In [ ]:
import numpy as np
import pandas as pd
import gc
import time
import csv
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, Callback
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Concatenate, Input, ReLU, MultiHeadAttention, Layer
import keras

from tensorflow.keras.losses import Loss
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from keras import backend as K

import matplotlib.pyplot as plt
import keras.backend as K

from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import joblib
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import re
from scipy.stats import rankdata, norm
from scipy.sparse import load_npz
import gc
import csv
import pickle
import glob
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import roc_auc_score
import numpy as np
import pickle


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#utility code

# Prepares a tsv file for submission from model output
class PredictionPreparation:
    def __init__(self, test_pred, Y_labels, protein_labels):
        self.test_pred = test_pred
        self.Y_labels = Y_labels
        self.protein_labels = protein_labels

    def prepare_submission(self, filename="submission.tsv", top_percent=1, dropweights=False, dropzeros=True):
        # prepare dataframe for submission
        df_finalSubmission_MF = pd.DataFrame(columns = ['Protein Id', 'GO Term Id','Prediction'])
        l = []
        for k in list(self.protein_labels):
            l += [k] * self.test_pred.shape[1]
        df_finalSubmission_MF['Protein Id'] = l
        df_finalSubmission_MF['GO Term Id'] = list(self.Y_labels) * self.test_pred.shape[0]
        df_finalSubmission_MF['Prediction'] = self.test_pred.ravel()

        # Calculate the number of rows that represent the top n%
        top_rows = int(top_percent * df_finalSubmission_MF.shape[0])

        # Select the top 5% rows based on 'Prediction'
        df_finalSubmission_MF = df_finalSubmission_MF.nlargest(top_rows, 'Prediction')

        if dropzeros:
            # Drop rows where 'Prediction' is equal or less than zero
            print('zeros dropped for', len(df_finalSubmission_MF[df_finalSubmission_MF['Prediction'] <= 0.0001]), 'rows')
            df_finalSubmission_MF = df_finalSubmission_MF[df_finalSubmission_MF['Prediction'] > 0.0001]


        # round to 3 decimals
        df_finalSubmission_MF['Prediction'] = df_finalSubmission_MF['Prediction'].round(3)

        if dropweights:
            # Drop the right most column (weights)
            df_finalSubmission_MF = df_finalSubmission_MF.iloc[:, :-1]
            print(df_finalSubmission_MF.head())
            print('matrix shape:', df_finalSubmission_MF.shape)

        else:
            # take a quick look at the data
            summary_stats = df_finalSubmission_MF['Prediction'].describe()
            print(df_finalSubmission_MF.head())
            print(summary_stats)

        # Save to file
        df_finalSubmission_MF.to_csv(filename, header=False, index=False, sep="\t")

# prepares training data depending on which ontology is being looked at
class TrainingDataPreparation:
    def __init__(self, ontology):
        self.ontology = ontology

    def load_Y_data(self):
        if self.ontology == 'BPO':
            Y = np.load('/content/drive/MyDrive/CAFA5 2023/Train_data/Y_BP_1500.npy', allow_pickle=True)
            Y_labels = np.load('/content/drive/MyDrive/CAFA5 2023/Train_data/Y_BP_labels_1500.npy', allow_pickle=True)
            df = pd.read_csv('/content/drive/MyDrive/CAFA5 2023/Train_data/BPO_1500_freq_weights.csv')
            weights_raw = df['IA_weight'].values.tolist()
            weights = {i: weights_raw[i] for i in range(len(weights_raw))}

        elif self.ontology == 'CCO':
            Y = np.load('/content/drive/MyDrive/CAFA5 2023/Train_data/Y_CC_800.npy', allow_pickle=True)
            Y_labels = np.load('/content/drive/MyDrive/CAFA5 2023/Train_data/Y_CC_labels_800.npy', allow_pickle=True)
            df = pd.read_csv('/content/drive/MyDrive/CAFA5 2023/Train_data/CCO_800_freq_weights.csv')
            weights_raw = df['IA_weight'].values.tolist()
            weights = {i: weights_raw[i] for i in range(len(weights_raw))}

        elif self.ontology == 'MFO':
            Y = np.load('/content/drive/MyDrive/CAFA5 2023/Train_data/Y_MF_800.npy', allow_pickle=True)
            Y_labels = np.load('/content/drive/MyDrive/CAFA5 2023/Train_data/Y_MF_labels_800.npy', allow_pickle=True)
            df = pd.read_csv('/content/drive/MyDrive/CAFA5 2023/Train_data/MFO_800_freq_weights.csv')
            weights_raw = df['IA_weight'].values.tolist()
            weights = {i: weights_raw[i] for i in range(len(weights_raw))}

        else:
            print('Error: ontology not recognized')
            return None, None, None
        print(f'Loaded {self.ontology} ontology')

        return Y, Y_labels, weights, df

    def load_t5_data(self):
        train_data1 = np.load('/content/drive/MyDrive/CAFA5 2023/Train_data/t5_data_sorted_f32.npy', mmap_mode='r')
        test_data1 = np.load('/content/drive/MyDrive/CAFA5 2023/Test_data/t5_data_sorted.npy', mmap_mode='r')

        print(f'T5 train data shape: {train_data1.shape}')
        print(f'T5 test data shape: {test_data1.shape}')

        return train_data1, test_data1

    def load_esm2_s_data(self):
        train_data2 = np.load('/content/drive/MyDrive/CAFA5 2023/Train_data/ems_data_sorted.npy', mmap_mode='r')
        test_data2 = np.load('/content/drive/MyDrive/CAFA5 2023/Test_data/ems_data_sorted.npy', mmap_mode='r')

        print(f'ESM2 small train data shape: {train_data2.shape}')
        print(f'ESM2 small test data shape: {test_data2.shape}')

        return train_data2, test_data2

    def load_esm2_l_data(self):
        train_data3 = np.load('/content/drive/MyDrive/CAFA5 2023/Train_data/ESM2_3B_train_embeddings_sorted.npy', mmap_mode='r')
        test_data3 = np.load('/content/drive/MyDrive/CAFA5 2023/Test_data/ESM2_3B_test_embeddings_sorted.npy', mmap_mode='r')

        print(f'ESM2 3B train data shape: {train_data3.shape}')
        print(f'ESM2 3B data shape: {test_data3.shape}')

        return train_data3, test_data3

    def load_pb_data(self):
        train_data4 = np.load('/content/drive/MyDrive/CAFA5 2023/Train_data/pb_data_sorted_f32.npy', mmap_mode='r')
        test_data4 = np.load('/content/drive/MyDrive/CAFA5 2023/Test_data/pb_data_sorted.npy', mmap_mode='r')

        print(f'PB train data shape: {train_data4.shape}')
        print(f'PB data shape: {test_data4.shape}')

        return train_data4, test_data4

    def load_ankh_data(self):
        train_data5 = np.load('/content/drive/MyDrive/CAFA5 2023/Train_data/Ankh_train_embeddings_sorted.npy', mmap_mode='r')
        test_data5 = np.load('/content/drive/MyDrive/CAFA5 2023/Test_data/Ankh_test_embeddings_sorted.npy', mmap_mode='r')

        print(f'Ankh train data shape: {train_data5.shape}')
        print(f'Ankh data shape: {test_data5.shape}')

        return train_data5, test_data5

    def load_taxa_data(self):
        train_data6 = np.load('/content/drive/MyDrive/CAFA5 2023/Train_data/protein_taxa_matrix_train.npy', mmap_mode='r')
        test_data6 = np.load('/content/drive/MyDrive/CAFA5 2023/Test_data/protein_taxa_matrix_test.npy', mmap_mode='r')

        print(f'Taxa train data shape: {train_data6.shape}')
        print(f'Taxa data shape: {test_data6.shape}')

        return train_data6, test_data6

    def load_text_embed(self):

        train_data7 = np.load('/content/drive/MyDrive/CAFA5 2023/Absrtract_embeds_modified/abstract_embeds_train_sorted_all.npy', mmap_mode='r')
        test_data7 = np.load('/content/drive/MyDrive/CAFA5 2023/Absrtract_embeds_modified/abstract_embeds_test_sorted_all.npy', mmap_mode='r')

        print(f'Text abstract data shape: {train_data7.shape}')
        print(f'Text abstract shape: {test_data7.shape}')

        return train_data7, test_data7

class PredictionPreparation:
    def __init__(self, test_pred, Y_labels, protein_labels):
        self.test_pred = test_pred
        self.Y_labels = Y_labels
        self.protein_labels = protein_labels

    def prepare_submission(self, filename="submission.tsv", top_percent=1, dropweights=False, dropzeros=True):
        # prepare dataframe for submission
        df_finalSubmission_MF = pd.DataFrame(columns = ['Protein Id', 'GO Term Id','Prediction'])
        l = []
        for k in list(self.protein_labels):
            l += [k] * self.test_pred.shape[1]
        df_finalSubmission_MF['Protein Id'] = l
        df_finalSubmission_MF['GO Term Id'] = list(self.Y_labels) * self.test_pred.shape[0]
        df_finalSubmission_MF['Prediction'] = self.test_pred.ravel()

        # Calculate the number of rows that represent the top n%
        top_rows = int(top_percent * df_finalSubmission_MF.shape[0])

        # Select the top 5% rows based on 'Prediction'
        df_finalSubmission_MF = df_finalSubmission_MF.nlargest(top_rows, 'Prediction')

        if dropzeros:
            # Drop rows where 'Prediction' is equal or less than zero
            print('zeros dropped for', len(df_finalSubmission_MF[df_finalSubmission_MF['Prediction'] <= 0.0001]), 'rows')
            df_finalSubmission_MF = df_finalSubmission_MF[df_finalSubmission_MF['Prediction'] > 0.0001]


        # round to 3 decimals
        df_finalSubmission_MF['Prediction'] = df_finalSubmission_MF['Prediction'].round(3)

        if dropweights:
            # Drop the right most column (weights)
            df_finalSubmission_MF = df_finalSubmission_MF.iloc[:, :-1]
            print(df_finalSubmission_MF.head())
            print('matrix shape:', df_finalSubmission_MF.shape)

        else:
            # take a quick look at the data
            summary_stats = df_finalSubmission_MF['Prediction'].describe()
            print(df_finalSubmission_MF.head())
            print(summary_stats)

        # Save to file
        df_finalSubmission_MF.to_csv(filename, header=False, index=False, sep="\t")

class ProteinPredictions:
    # Initialize an empty dictionary to store the predictions
    def __init__(self):
        self.predictions = {}

    # Add a prediction to the storage, with optional bonus
    # Arguments:
    #   - protein: Identifier for the protein
    #   - go_term: GO term that is being predicted
    #   - score: Confidence score of the prediction
    #   - branch: Branch of the Gene Ontology (e.g., 'CCO', 'MFO', 'BPO')
    #   - bonus: Optional bonus to be added to the score
    def add_prediction(self, protein, go_term, score, branch, bonus=1, adjustment=1):
        # If the protein is not already in the storage, initialize its structure
        if protein not in self.predictions:
            self.predictions[protein] = {'CCO': {}, 'MFO': {}, 'BPO': {}}

        # Convert the score to a float for comparison and calculation
        score = float(score)

        # If this GO term has already been predicted for this protein and branch,
        # add the bonus to the score. Keep the highest score.
        if go_term in self.predictions[protein][branch]:
            self.predictions[protein][branch][go_term] *= 1+(score**3)*bonus
            self.predictions[protein][branch][go_term] += score*adjustment

        # If this GO term has not been predicted yet, store it with the score
        else:
            self.predictions[protein][branch][go_term] = max(score*adjustment, 0)

        # Ensure that the score does not exceed 1
        if self.predictions[protein][branch][go_term] > 1:
            self.predictions[protein][branch][go_term] = 1

    # Get a list of all scores in the predictions
    def get_scores(self):
        scores = []
        for protein, branches in self.predictions.items():
            for branch, go_terms in branches.items():
                scores.extend(go_terms.values())
        return scores

    def plot_predictions(self):
        scores = self.get_scores()
        plt.hist(scores, bins=30, edgecolor='black')
        plt.title('Distribution of Prediction Scores')
        plt.xlabel('Score')
        plt.ylabel('Frequency')
        plt.show()

    # Export the stored predictions to a file
    # Arguments:
    #   - output_file: File name for the exported predictions
    #   - top: Number of top predictions to export for each protein and branch
    def get_predictions(self, output_file='submission.tsv', top=60):
        # Open the output file
        with open(output_file, 'w') as f:
            # Iterate through each protein and its branches
            for protein, branches in self.predictions.items():
                # For each branch, sort the GO terms by score in descending order and select the top ones
                for branch, go_terms in branches.items():
                    # Sort go_terms by score in descending order and take the top ones
                    top_go_terms = sorted(go_terms.items(), key=lambda x: x[1], reverse=True)[:top]
                    # Write each of the top predictions to the file
                    for go_term, score in top_go_terms:
                        f.write(f"{protein}\t{go_term}\t{score:.3f}\n")


def extract_go_terms_and_branches(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
        # Match each stanza with [Term] in the OBO file
        stanzas = re.findall(r'\[Term\][\s\S]*?(?=\n\[|$)', content)

    go_terms_dict = {}
    for stanza in stanzas:
        # Extract the GO term ID
        go_id = re.search(r'^id: (GO:\d+)', stanza, re.MULTILINE)
        if go_id:
            go_id = go_id.group(1)

        # Extract the namespace (branch)
        namespace = re.search(r'^namespace: (\w+)', stanza, re.MULTILINE)
        if namespace:
            namespace = namespace.group(1)

        if go_id and namespace:
            # Map the branch abbreviation to the corresponding BPO, CCO, or MFO
            branch_abbr = {'biological_process': 'BPO', 'cellular_component': 'CCO', 'molecular_function': 'MFO'}
            go_terms_dict[go_id] = branch_abbr[namespace]

    return go_terms_dict

def get_constr_out(x, R):
    """ Given the output of the neural network x returns the output of MCM given the hierarchy constraint expressed in the matrix R """
    c_out = tf.cast(x, tf.float32)
    print('c_out1', c_out.shape)
    c_out = tf.expand_dims(c_out, 1)
    print('c_out2', c_out.shape)
    c_out = tf.tile(c_out, [1, tf.shape(R)[1], 1])
    print('c_out3', c_out.shape)
    # c_out = tf.expand_dims(c_out, -2)  # Expand the last dimension of c_out
    # print('c_out2', c_out.shape)
    # c_out = tf.tile(c_out, [1, tf.shape(R)[1]], 1)  # Make c_out match the shape of R
    # print('c_out3', c_out.shape)
    R = tf.cast(R, tf.float32)
    print('R', R.shape)
    R = tf.expand_dims(R, axis=0)  # make R 3D by adding an extra dimension
    print('R', R.shape)
    R_batch = tf.tile(R, [tf.shape(x)[0], 1, 1])  # replicate R along the batch dimension
    print('R_batch', R_batch.shape)
    print(R_batch[0][50][0:50])
    print(c_out[0][0][:50])
    final_out = tf.reduce_max(R_batch * c_out, axis=2)
    print('final_out', final_out.shape)
    return final_out

class CustomMCM(Loss):
    def __init__(self, R, name="custom_mcm"):
        super().__init__(name=name)
        self.R = tf.cast(R, tf.float32)

    def call(self, y_true, y_pred):
        constr_output = get_constr_out(y_pred, self.R)
        y_true = tf.cast(y_true, tf.float32)
        print('yt, yp, c_out', y_true.shape, y_pred.shape, constr_output.shape)
        train_output = y_true*y_pred
        print('train_output1', train_output.shape)
        train_output = get_constr_out(train_output, self.R)
        print('train_output2', train_output.shape)
        train_output = (1-y_true)*constr_output + y_true*train_output
        print('train_output3', train_output.shape)
        # return binary cross entropy loss
        #return K.mean(K.binary_crossentropy(y_true, train_output), axis=-1) #this does not make things better -1 -> -2
        return tf.keras.losses.binary_crossentropy(y_true, train_output)


file_path = '/content/drive/MyDrive/CAFA5 2023/Train_data/go-basic.obo'
go_terms_dict = extract_go_terms_and_branches(file_path)


In [ ]:
ontology = 'MFO'
data_prep = TrainingDataPreparation(ontology)
Y, Y_labels, weights, df = data_prep.load_Y_data()
t5_train, t5_test = data_prep.load_t5_data()
esm1_train, esm1_test = data_prep.load_esm2_s_data()
esm2_train, esm2_test = data_prep.load_esm2_l_data()
pb_train, pb_test = data_prep.load_pb_data()
ankh_train, ankh_test = data_prep.load_ankh_data()
taxa_train, taxa_test = data_prep.load_taxa_data()
txt1_train, txt1_test = data_prep.load_text_embed()

if ontology == 'CCO':
    R = np.load('/content/drive/MyDrive/CAFA5 2023/Train_data/CCO_800_DAG_matrix.npy')
    output_split = [4, 11, 24, 59, 199]

elif ontology == 'MFO':
    R = np.load('/content/drive/MyDrive/CAFA5 2023/Train_data/MFO_800_DAG_matrix.npy')
    output_split = [4,14,28,69,176]

elif ontology == 'BPO':
    R = np.load('/content/drive/MyDrive/CAFA5 2023/Train_data/BPO_1500_DAG_matrix.npy')
    output_split = [13,45,102,213,465]


print (f'R shape is {R.shape}')
print('datasets loaded')

Loaded MFO ontology
T5 train data shape: (142246, 1024)
T5 test data shape: (141864, 1024)
ESM2 small train data shape: (142246, 1280)
ESM2 small test data shape: (141864, 1280)
ESM2 3B train data shape: (142246, 2560)
ESM2 3B data shape: (141864, 2560)
PB train data shape: (142246, 1024)
PB data shape: (141864, 1024)
Ankh train data shape: (142246, 1536)
Ankh data shape: (141864, 1536)
Taxa train data shape: (142246, 70)
Taxa data shape: (141864, 70)
Text abstract data shape: (142246, 10279)
Text abstract shape: (141864, 10279)
R shape is (800, 800)
datasets loaded


In [ ]:
print(f'Number of proteins with no {ontology} terms = {np.sum(np.all(Y == 0, axis=1))} out of {Y.shape[0]}')

valid_rows = ~np.all(Y == 0, axis=1)   #~ inverts np all
# Filter out rows in Y
Y = Y[valid_rows]

# Filter out rows in other input arrays
t5_train = t5_train[valid_rows]
esm1_train = esm1_train[valid_rows]
esm2_train = esm2_train[valid_rows]
pb_train = pb_train[valid_rows]
ankh_train = ankh_train[valid_rows]
taxa_train = taxa_train[valid_rows]
txt1_train = txt1_train[valid_rows]

print(f'we retained {np.sum(valid_rows)} proteins from training set where {ontology} terms are present')

Number of proteins with no MFO terms = 63114 out of 142246
we retained 79132 proteins from training set where MFO terms are present


In [ ]:
# Setup vars
n_epochs = 20
n_epochs_ft = 10
dropout_rate = 0.5
l1_dim = 600
l2_dim = 300
final_dim = 800

txt_dim1 = 200
txt_dim2 = 200

#loss without removing any training data (proteins with no BPO...) = 0.0310

#txt dim tests with BPO loss numbers
#no text input = 0.04613
#600/300 = 0.04313
#400/200 = 0.04302
#300/150 = 0.04295
#200/200 = 0.04280
  #final_dim 800 -> 400 = 0.04343
  #final dense layer removed completely -> 0.04286
#200/100 = 0.04302
#140/140 = 0.04306

#modifying the l1_dim 600 -> 300 = 0.04297
#l2_dim 300->450 = 0.04281
#longer training txt dim 100/100 = 0.04309

In [ ]:
rand_state_no = 1
#output_split = [10,20,30,40,50]
output_split = [5,10,20,40,80]

# Define 5-fold cross validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=rand_state_no)

fold_ct = 1
# Iterate over each fold
for train_index, val_index in kf.split(t5_train):

    # Skip other folds for testing
    # if fold_ct>1:
    #   continue

    # Define save name
    save_name = '/content/drive/MyDrive/CAFA5 2023/1.30/'+f"{ontology}_fc{fold_ct}_r{rand_state_no}_1.30.4"

    x_tr_1, x_val_1 = t5_train[train_index], t5_train[val_index]
    x_tr_2, x_val_2 = esm1_train[train_index], esm1_train[val_index]
    x_tr_3, x_val_3 = esm2_train[train_index], esm2_train[val_index]
    x_tr_4, x_val_4 = pb_train[train_index], pb_train[val_index]
    x_tr_5, x_val_5 = ankh_train[train_index], ankh_train[val_index]
    x_tr_6, x_val_6 = taxa_train[train_index], taxa_train[val_index]
    x_tr_7, x_val_7 = txt1_train[train_index], txt1_train[val_index]
    # x_tr_8, x_val_8 = txt2_train[train_index], txt2_train[val_index]
    # x_tr_9, x_val_9 = txt3_train[train_index], txt3_train[val_index]

    y_tr_1, y_val_1 = Y[train_index,:output_split[0]], Y[val_index,:output_split[0]]
    y_tr_2, y_val_2 = Y[train_index,output_split[0]:output_split[1]], Y[val_index,output_split[0]:output_split[1]]
    y_tr_3, y_val_3 = Y[train_index,output_split[1]:output_split[2]], Y[val_index,output_split[1]:output_split[2]]
    y_tr_4, y_val_4 = Y[train_index,output_split[2]:output_split[3]], Y[val_index,output_split[2]:output_split[3]]
    y_tr_5, y_val_5 = Y[train_index,output_split[3]:output_split[4]], Y[val_index,output_split[3]:output_split[4]]
    y_tr_6, y_val_6 = Y[train_index,output_split[4]:], Y[val_index,output_split[4]:]

    # Define the inputs
    t5_in = Input(shape=(t5_train.shape[1],))
    esm1_in = Input(shape=(esm1_train.shape[1],))
    mlpr_in = Input(shape=(esm2_train.shape[1],))
    knn_in = Input(shape=(pb_train.shape[1],))
    lr_in = Input(shape=(ankh_train.shape[1],))
    taxa_in = Input(shape=(taxa_train.shape[1],))
    txt1_in = Input(shape=(txt1_train.shape[1],))
    # txt2_in = Input(shape=(txt2_train.shape[1],))
    # txt3_in = Input(shape=(txt3_train.shape[1],))

    network_1 = Dense(l1_dim, name='t5_dense_1')(t5_in)
    network_1 = BatchNormalization(name='t5_batchnorm_1')(network_1)
    network_1 = LeakyReLU(alpha=0.1, name='t5_leakyrelu_1')(network_1)
    network_1 = Dropout(dropout_rate, name='t5_dropout_1')(network_1)
    network_1 = Dense(l2_dim, name='t5_dense_2')(network_1)

    network_2 = Dense(l1_dim, name='esm1_dense_1')(esm1_in)
    network_2 = BatchNormalization(name='esm1_batchnorm_1')(network_2)
    network_2 = LeakyReLU(alpha=0.1, name='esm1_leakyrelu_1')(network_2)
    network_2 = Dropout(dropout_rate, name='esm1_dropout_1')(network_2)
    network_2 = Dense(l2_dim, name='esm1_dense_2')(network_2)

    network_3 = Dense(l1_dim, name='mlpr_dense_1')(mlpr_in)
    network_3 = BatchNormalization(name='mlpr_batchnorm_1')(network_3)
    network_3 = LeakyReLU(alpha=0.1, name='mlpr_leakyrelu_1')(network_3)
    network_3 = Dropout(dropout_rate, name='mlpr_dropout_1')(network_3)
    network_3 = Dense(l2_dim, name='mlpr_dense_2')(network_3)

    network_4 = Dense(l1_dim, name='knn_dense_1')(knn_in)
    network_4 = BatchNormalization(name='knn_batchnorm_1')(network_4)
    network_4 = LeakyReLU(alpha=0.1, name='knn_leakyrelu_1')(network_4)
    network_4 = Dropout(dropout_rate, name='knn_dropout_1')(network_4)
    network_4 = Dense(l2_dim, name='knn_dense_2')(network_4)

    network_5 = Dense(l1_dim, name='lr_dense_1')(lr_in)
    network_5 = BatchNormalization(name='lr_batchnorm_1')(network_5)
    network_5 = LeakyReLU(alpha=0.1, name='lr_leakyrelu_1')(network_5)
    network_5 = Dropout(dropout_rate, name='lr_dropout_1')(network_5)
    network_5 = Dense(l2_dim, name='lr_dense_2')(network_5)

    network_6 = Dense(128, name='taxa_dense_1')(taxa_in)
    network_6 = BatchNormalization(name='taxa_batchnorm_1')(network_6)
    network_6 = LeakyReLU(alpha=0.1, name='taxa_leakyrelu_1')(network_6)
    network_6 = Dropout(dropout_rate, name='taxa_dropout_1')(network_6)
    network_6 = Dense(64, name='taxa_dense_2')(network_6)

    network_7 = Dense(txt_dim1, name='txt1_dense_1')(txt1_in)
    network_7 = BatchNormalization(name='txt1_batchnorm_1')(network_7)
    network_7 = LeakyReLU(alpha=0.1, name='txt1_leakyrelu_1')(network_7)
    network_7 = Dropout(dropout_rate, name='txt1_dropout_1')(network_7)
    network_7 = Dense(txt_dim2, name='txt1_dense_2')(network_7)

    # Concatenate the networks
    combined = Concatenate()([network_1, network_2, network_3, network_4, network_5, network_6, network_7])
    #combined = LeakyReLU(alpha=0.1, name='combined_leakyrelu_1')(combined)
    combined = BatchNormalization(name = 'combined_batchnorm_1')(combined)
    combined = Dense(final_dim, name='combined_dense_1', activation='relu')(combined)

    # First level output
    output_1 = Dense(output_split[0], activation='sigmoid', name='final_output_1_112')(combined)

    # Second level output, connected to first level
    combined_2 = Concatenate()([combined, output_1])
    output_2 = Dense(output_split[1]-output_split[0], activation='sigmoid', name='final_output_2_112')(combined_2)

    # Third level output, connected to second level
    combined_3 = Concatenate()([combined, output_1, output_2])
    output_3 = Dense(output_split[2]-output_split[1], activation='sigmoid', name='final_output_3_112')(combined_3)

    # Fourth level output, connected to third level
    combined_4 = Concatenate()([combined, output_1, output_2, output_3])
    output_4 = Dense(output_split[3]-output_split[2], activation='sigmoid', name='final_output_4_112')(combined_4)

    # Fifth level output, connected to fourth level
    combined_5 = Concatenate()([combined, output_1, output_2, output_3, output_4])
    output_5 = Dense(output_split[4]-output_split[3], activation='sigmoid', name='final_output_5_112')(combined_5)

    combined_6 = Concatenate()([combined, output_1, output_2, output_3, output_4, output_5])
    output_6 = Dense(Y.shape[1]-output_split[4], activation='sigmoid', name='final_output_6_112')(combined_6)

    # Create the model
    model = Model(inputs=[t5_in, esm1_in, mlpr_in, knn_in, lr_in, taxa_in, txt1_in],
                  outputs=[output_1, output_2, output_3, output_4, output_5, output_6])

    #lr scheduling
    def lr_schedule(epoch, lr):
        if epoch > 0 and epoch % 7 == 0:
            lr = lr * 0.5
        return lr

    # Define callbacks
    checkpoint_loss = ModelCheckpoint(f'{save_name}.hdf5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='min')
    csv_logger = CSVLogger(f'{save_name}.csv')
    lr_callback = LearningRateScheduler(lr_schedule, verbose=1)
    weights_list = df['IA_weight'].values.tolist()

    #Add weights from IA
    weights_1 = weights_list[:output_split[0]]
    weights_2 = weights_list[output_split[0]:output_split[1]]
    weights_3 = weights_list[output_split[1]:output_split[2]]
    weights_4 = weights_list[output_split[2]:output_split[3]]
    weights_5 = weights_list[output_split[3]:output_split[4]]
    weights_6 = weights_list[output_split[4]:]

    # Compile model
    model.compile(optimizer=Adam(learning_rate=0.0003),
    loss={'final_output_1_112': 'binary_crossentropy',
          'final_output_2_112': 'binary_crossentropy',
          'final_output_3_112': 'binary_crossentropy',
          'final_output_4_112': 'binary_crossentropy',
          'final_output_5_112': 'binary_crossentropy',
          'final_output_6_112': 'binary_crossentropy'},
    loss_weights={'final_output_1_112': weights_1,
          'final_output_2_112': weights_2,
          'final_output_3_112': weights_3,
          'final_output_4_112': weights_4,
          'final_output_5_112': weights_5,
          'final_output_6_112': weights_6},
    metrics=['binary_accuracy', tf.keras.metrics.AUC()])

    # Train model
    history = model.fit([x_tr_1, x_tr_2, x_tr_3, x_tr_4, x_tr_5, x_tr_6, x_tr_7], [y_tr_1, y_tr_2, y_tr_3, y_tr_4, y_tr_5, y_tr_6],
              validation_data=([x_val_1, x_val_2, x_val_3, x_val_4, x_val_5, x_val_6, x_val_7], [y_val_1, y_val_2, y_val_3, y_val_4, y_val_5, y_val_6]),
              epochs=n_epochs, batch_size=256,
              callbacks=[lr_callback, checkpoint_loss, csv_logger])

    # Code helth
    K.clear_session()
    gc.collect()

    fold_ct +=1


Epoch 1: LearningRateScheduler setting learning rate to 0.0003000000142492354.
Epoch 1/20
248/248 [==============================] - ETA: 0s - loss: 1.9243 - final_output_1_112_loss: 0.2927 - final_output_2_112_loss: 0.1947 - final_output_3_112_loss: 0.2023 - final_output_4_112_loss: 0.1249 - final_output_5_112_loss: 0.0876 - final_output_6_112_loss: 0.0513 - final_output_1_112_binary_accuracy: 0.8781 - final_output_1_112_auc: 0.9341 - final_output_2_112_binary_accuracy: 0.9362 - final_output_2_112_auc: 0.8022 - final_output_3_112_binary_accuracy: 0.9212 - final_output_3_112_auc: 0.9229 - final_output_4_112_binary_accuracy: 0.9633 - final_output_4_112_auc: 0.7693 - final_output_5_112_binary_accuracy: 0.9713 - final_output_5_112_auc: 0.8673 - final_output_6_112_binary_accuracy: 0.9847 - final_output_6_112_auc: 0.6872
Epoch 1: val_loss improved from inf to 2.10678, saving model to /content/drive/MyDrive/CAFA5 2023/1.30/MFO_fc1_r1_1.30.4.hdf5
248/248 [==============================] - 29

In [ ]:
# For submission for CAFA test set
test_labels = np.load('/content/drive/MyDrive/CAFA5 2023/Test_data/t5_labels_sorted.npy')
rand_state_nos = [1,2,3,4,5]

predictions = []

for rand_state_no in rand_state_nos:

    for fold_ct in [1,2,3,4,5]:

        model_path = f'/content/drive/MyDrive/CAFA5 2023/1.30/{ontology}_fc{str(fold_ct)}_r{rand_state_no}_1.30.1.hdf5'
        #model.load_weights(model_path) #only for 1.24.2
        model = load_model(model_path)

        # Add OOF predictions for this split
        predicted_outputs = model.predict([t5_test, esm1_test, esm2_test, pb_test, ankh_test, taxa_test, txt1_test])
        #output_1, output_2, output_3, output_4, output_5, output_6 = model.predict([x_val_1, x_val_2, x_val_3, x_val_4, x_val_5, x_val_6]) #only for 1.24.3
        #predicted_outputs = np.concatenate([output_1, output_2, output_3, output_4, output_5, output_6], axis=1) #only for 1.24.3

        predictions.append(predicted_outputs)

        K.clear_session()
        gc.collect()

average_predictions = np.mean(predictions, axis=0)
print('average predictions shape:', average_predictions.shape)

prediction_preparation = PredictionPreparation(average_predictions, Y_labels, test_labels)
prediction_preparation.prepare_submission(filename=f"/content/drive/MyDrive/CAFA5 2023/1.30/{ontology}_test_submission_1.30.1.tsv", top_percent=0.05)


4434/4434 [==============================] - 14s 3ms/step
average predictions shape: (141864, 800)
zeros dropped for 0 rows
          Protein Id  GO Term Id  Prediction
100647203     Q9FPJ8  GO:0005488         1.0
108245603     Q9SX79  GO:0005488         1.0
90861603      Q8VXZ9  GO:0005488         1.0
106876803     Q9S7N9  GO:0005488         1.0
107233603     Q9SG10  GO:0005488         1.0
count    5.674560e+06
mean     1.280636e-01
std      2.524600e-01
min      6.000000e-03
25%      1.000000e-02
50%      1.900000e-02
75%      7.400000e-02
max      1.000000e+00
Name: Prediction, dtype: float64


In [ ]:
# For submission for CAFA test set 130b
test_labels = np.load('/content/drive/MyDrive/CAFA5 2023/Test_data/t5_labels_sorted.npy')
model_nos = [2,3]
rand_state_no = 1

predictions = []

for model_no in model_nos:

    for fold_ct in [1,2,3,4,5]:

        model_path = f'/content/drive/MyDrive/CAFA5 2023/1.30/{ontology}_fc{str(fold_ct)}_r{rand_state_no}_1.30.{model_no}.hdf5'
        #model.load_weights(model_path) #only for 1.24.2
        model = load_model(model_path)

        # Add OOF predictions for this split
        #predicted_outputs = model.predict([t5_test, esm1_test, esm2_test, pb_test, ankh_test, taxa_test, txt1_test])
        output_1, output_2, output_3, output_4, output_5, output_6 = model.predict([t5_test, esm1_test, esm2_test, pb_test, ankh_test, taxa_test, txt1_test]) #only for 1.24.3
        predicted_outputs = np.concatenate([output_1, output_2, output_3, output_4, output_5, output_6], axis=1) #only for 1.24.3

        predictions.append(predicted_outputs)

        K.clear_session()
        gc.collect()

average_predictions = np.mean(predictions, axis=0)
print('average predictions shape:', average_predictions.shape)

prediction_preparation = PredictionPreparation(average_predictions, Y_labels, test_labels)
prediction_preparation.prepare_submission(filename=f"/content/drive/MyDrive/CAFA5 2023/1.30/{ontology}_test_submission_1.30b.tsv", top_percent=0.05)


InternalError: ignored

In [ ]:
# Make groundtruth file for evaluating against
np.random.seed(42)  # for reproducibility

train_labels = np.load('/content/drive/MyDrive/CAFA5 2023/Train_data/Ankh_train_labels_sorted.npy')

# Set the number of proteins to use as testing for ground truth
test_size = 1002

# Create groundtruth data for testing
random_indices = np.random.choice(t5_train.shape[0], size=test_size, replace=False)
random_labels = train_labels[random_indices]
ramdom_Y = Y[random_indices]

print(f'Check random indices match: {random_indices[:5]}')

prediction_preparation = PredictionPreparation(ramdom_Y, Y_labels, random_labels)
prediction_preparation.prepare_submission(filename=f"/content/drive/MyDrive/CAFA5 2023/1.30/Groundtruth_{test_size}_{ontology}_submission.tsv", top_percent=1, dropweights=True)


Check random indices match: [35520 76725 73256 45903 50891]
zeros dropped for 795517 rows
        Protein Id  GO Term Id
400800      B2RYH7  GO:0003824
612800      P0C1T1  GO:0003824
640603  A0A5F9ZH03  GO:1902936
12801       C9JMY5  GO:0097159
12802       C9JMY5  GO:1901363
matrix shape: (6083, 2)


In [ ]:
# Internal testing for 1.30.2/3/4
train_labels = np.load('/content/drive/MyDrive/CAFA5 2023/Train_data/Ankh_train_labels_sorted.npy')
test_labels = np.load('/content/drive/MyDrive/CAFA5 2023/Test_data/t5_labels_sorted.npy')

# change for each set of models to be evaluated
eval_no = 3

np.random.seed(42)  # for reproducibility
test_size = 1002

model_nos = [2]
rand_state_no = 1

# Initialize array to accumulate OOF predictions across evaluated models
total_oof_predictions = np.zeros_like(Y, dtype=np.float32)

for model_no in model_nos:

    # Define 5-fold cross validation
    n_splits = 5
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=rand_state_no)

    fold_ct = 1
    out_of_fold_predictions = np.zeros_like(Y, dtype=np.float32) # Initialize array with same shape as Y, MUST BE FLOAT32 OR IT FAILS

    # Iterate over each fold and produce prediction on validation data only
    for train_index, val_index in kf.split(t5_train):

        x_tr_1, x_val_1 = t5_train[train_index], t5_train[val_index]
        x_tr_2, x_val_2 = esm1_train[train_index], esm1_train[val_index]
        x_tr_3, x_val_3 = esm2_train[train_index], esm2_train[val_index]
        x_tr_4, x_val_4 = pb_train[train_index], pb_train[val_index]
        x_tr_5, x_val_5 = ankh_train[train_index], ankh_train[val_index]
        x_tr_6, x_val_6 = taxa_train[train_index], taxa_train[val_index]
        x_tr_7, x_val_7 = txt1_train[train_index], txt1_train[val_index]


        model_path = f'/content/drive/MyDrive/CAFA5 2023/1.30/{ontology}_fc{str(fold_ct)}_r{rand_state_no}_1.30.{model_no}.hdf5'
        #model.load_weights(model_path) #only for 1.24.2
        model = load_model(model_path)

        # Add OOF predictions for this split
        #predicted_outputs = model.predict([x_val_1, x_val_2, x_val_3, x_val_4, x_val_5, x_val_6, x_val_7])
        output_1, output_2, output_3, output_4, output_5, output_6 = model.predict([x_val_1, x_val_2, x_val_3, x_val_4, x_val_5, x_val_6, x_val_7]) #only for 1.24.3
        predicted_outputs = np.concatenate([output_1, output_2, output_3, output_4, output_5, output_6], axis=1) #only for 1.24.3

        out_of_fold_predictions[val_index, :] = predicted_outputs

        K.clear_session()
        gc.collect()

        fold_ct+=1

    total_oof_predictions += out_of_fold_predictions
averaged_oof_predictions = total_oof_predictions / len(model_nos)


random_indices = np.random.choice(t5_train.shape[0], size=test_size, replace=False)

predicted_output = out_of_fold_predictions[random_indices]
random_labels = train_labels[random_indices]
ramdom_Y = Y[random_indices]

print(f'Check random indices match: {random_indices[:5]}')
print(f'Output predictions shape:{predicted_output.shape}')

prediction_preparation = PredictionPreparation(predicted_output, Y_labels, random_labels)
prediction_preparation.prepare_submission(filename=f"/content/drive/MyDrive/CAFA5 2023/1.30/{ontology}_testing_1.30b.{eval_no}.tsv", top_percent=0.05)

# Code helth
K.clear_session()
gc.collect()


#1.30b.5 = 2,3,4 models ensemble
#1.30b.6 = 2,3 ensemble
#1.30b.7 = just 2

495/495 [==============================] - 2s 3ms/step
Check random indices match: [35520 76725 73256 45903 50891]
Output predictions shape:(1002, 800)
zeros dropped for 0 rows
       Protein Id  GO Term Id  Prediction
402403     O66874  GO:0005488         1.0
700803     P36152  GO:0005488         1.0
571203     P76010  GO:0005488         1.0
608803     P07334  GO:0005488         1.0
772800     G3V2W0  GO:0003824         1.0
count    40080.000000
mean         0.152165
std          0.256974
min          0.016000
25%          0.023000
50%          0.040000
75%          0.112000
max          1.000000
Name: Prediction, dtype: float64


110